# KFC Healthy diagnose
Sends 3 heart beat signals to concatenated kfcs, two for either and one through both.
The pulses received will be used to diagnose the healthy of the system.

In [1]:
from kafka import KafkaProducer,KafkaConsumer
import threading, logging, time, Queue
from datetime import datetime
import multiprocessing



In [2]:
topics = ["beatsGlobal","beatsLocal"]
KFC_0 = ['smartaqnet-0-node1:9092','smartaqnet-0-node2:9092','smartaqnet-0-node3:9092']
KFC_1 = ['smartaqnet-1-node1:9092']
#

global heartBeatCounter
global heartBeatMessage
heartBeatCounter = [0,0,0]
heartBeatMessage = 0

In [3]:
heartBeatCounter

[0, 0, 0]

In [4]:
class HeartbeatGenerator(multiprocessing.Process):
    def __init__(self, KFC_Main, KFC_Backup, topics, beatRate = 15, printLog = True):
        '''
        Send heartbeats every 15 Sec
        '''
        multiprocessing.Process.__init__(self)
        self.stop_event = multiprocessing.Event()
        self.KFC_Main = KFC_Main
        self.KFC_Backup = KFC_Backup
        self.topics = topics
        self.printLog = printLog
        self.beatRate = beatRate
    def stop(self):
        self.stop_event.set()
        
    def run(self):
        global heartBeatCounter
        global heartBeatMessage
        # Send heart beats to KFC0 to both topics
        producer_KFC_Main = KafkaProducer(bootstrap_servers= self.KFC_Main)
        # Send heart beats 
        producer_KFC_Backup = KafkaProducer(bootstrap_servers= self.KFC_Backup)
        
        while not self.stop_event.is_set():
            message = str(heartBeatMessage) #datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            if(printLog):
                print("Sending global heartbeat:" + message)
                
            producer_KFC_Main.send(topic = topics[0], value = message)
            heartBeatCounter[0] += 1
            if(printLog):
                print("Sending local heartbeat to KFC0:" + message)
            producer_KFC_Main.send(topic = topics[1], value = message, key = "KFC_Main")
            heartBeatCounter[1] += 1
            if(printLog):
                print("Sending local heartbeat to KFC1:" + message)
            producer_KFC_Backup.send(topic = topics[1], value = message , key = "KFC_Backup")
            heartBeatCounter[2] += 1
            time.sleep(self.beatRate)
            #print(heartBeatCounter)
            #print(heartBeatMessage)
            #print("#####")
            heartBeatMessage += 1
            if self.stop_event.is_set():
                break

class HeartRateMonitor(multiprocessing.Process):
    def __init__(self, KFC, topics, printLog = False):
        multiprocessing.Process.__init__(self)
        self.stop_event = multiprocessing.Event()
        self.KFC = KFC
        self.topics = topics
        self.printLog = printLog
    def stop(self):
        self.stop_event.set()
        
    def run(self):
        global heartBeatCounter
        global heartBeatMessage
        consumer_KFC = KafkaConsumer(bootstrap_servers= self.KFC,
                                 auto_offset_reset='latest')
        consumer_KFC.subscribe(topics)
        while not self.stop_event.is_set():
            for message in consumer_KFC:
                i = -1
                if(str(message.topic) == "beatsGlobal"):
                    i = 0
                elif(str(message.topic) == "beatsLocal"):
                    if(message.key == "KFC_Main"):
                        i = 1
                    elif(message.key == "KFC_Backup"):
                        i = 2
                if(i >= 0):
                    print("heartBeatMessage:{},message.value:{},newValue at {} = {}".format(heartBeatMessage,message.value,i,max(0, heartBeatMessage - int(message.value))))
                    heartBeatCounter[i] = max(0, heartBeatMessage - int(message.value))
                print(heartBeatCounter)
                if(printLog):
                    print("Receive {} message from {}: {}".format(str(message.topic), self.KFC, str(message.value)))
                if self.stop_event.is_set():
                    break

        consumer.close()

In [5]:
printLog = True
beatRate = 5
tasks = [
        HeartbeatGenerator(KFC_0, KFC_1, topics, beatRate = beatRate, printLog = printLog),
    HeartRateMonitor(KFC_0 , topics, printLog = printLog),
    HeartRateMonitor(KFC_1, topics, printLog= printLog)
]

In [6]:
for t in tasks:
        t.start()

Process HeartRateMonitor-3:
Traceback (most recent call last):
  File "/home/wang/anaconda2/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    self.run()
  File "<ipython-input-4-d1299a98dd0a>", line 61, in run
    auto_offset_reset='latest')
  File "/home/wang/anaconda2/lib/python2.7/site-packages/kafka/consumer/group.py", line 324, in __init__
    self._client = KafkaClient(metrics=self._metrics, **self.config)
  File "/home/wang/anaconda2/lib/python2.7/site-packages/kafka/client_async.py", line 221, in __init__
    self.config['api_version'] = self.check_version(timeout=check_timeout)
  File "/home/wang/anaconda2/lib/python2.7/site-packages/kafka/client_async.py", line 826, in check_version
    raise Errors.NoBrokersAvailable()
NoBrokersAvailable: NoBrokersAvailable
Process HeartbeatGenerator-1:
Traceback (most recent call last):
  File "/home/wang/anaconda2/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    self.run()
  File "<ipython-input-4

In [53]:
for t in tasks:
        t.stop()

In [1]:
mystatus = [0,0,0]

In [2]:
mystatus[0] +=1

In [4]:
mystatus

[1, 0, 0]